In [188]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
import string
import warnings
import re

In [189]:
f = open("HarryPotterCh1_SorcererStone.txt","r") 
textbook = f.read()
#print(textbook)
f.close()

Here, we create our own function for preprocessing, tokenization as well as creating index of the words.

In [190]:
def preprocess_text(string):
    string = re.sub(r"\n", " ", string)     
    string = string.lower()
    return string

In [191]:
text=preprocess_text(textbook)
text_words=text.split()
text_len=len(text_words)

In [192]:
#Create dictionary from list of words in text
def dictionary(words):
    #create list of words without their dupications 
    words=set(words)
    #map word to index
    indx = {key: i for i, key in enumerate(words)}
    return indx



In [193]:
words_index=dictionary(text_words)
words_index

{'"--and': 0,
 'ready!"': 1,
 'shouted,': 2,
 'wrought-iron': 3,
 'shouted.': 4,
 'both': 4554,
 'yellow': 5,
 'four': 6,
 'alley."': 5823,
 'unwrapped.': 7,
 'spiders': 8,
 'hanging': 9,
 'clearing.': 5511,
 'screaming.': 11,
 'fiddling': 9444,
 'wand."': 7417,
 '"howling': 13,
 '"ouch!"': 14,
 'wizardry': 15,
 'cake,': 7418,
 '"nothing?"': 17,
 "fluffy's": 18,
 'lord': 19,
 '"trust': 20,
 'flicking': 21,
 'three-thirty': 22,
 'animals."': 23,
 'sinking': 24,
 'figg': 25,
 'wood,': 27,
 'wood.': 28,
 'foul': 30,
 'pretended': 6847,
 'office?"': 31,
 'fingers.': 32,
 'fingers,': 33,
 'bringing': 200,
 'wood?': 35,
 'angry...': 6605,
 'disturb': 36,
 'wooden': 37,
 'wednesday': 38,
 'this..."': 39,
 'stars.': 241,
 'everything."': 41,
 'screaming!': 42,
 'countercurse,': 43,
 'screeched.': 44,
 'straight': 45,
 'stars!': 46,
 'seven,': 47,
 'shelves,': 48,
 'her.': 49,
 'tired': 50,
 'berserk.': 51,
 'excellent,"': 52,
 'bacon': 53,
 'known."': 54,
 'second': 55,
 'shelves;': 56,
 'admi

Create sequences of 11 length (given 10 words as inputs, predict 1 word for output)

In [194]:
seq_len=11

In [195]:
def  create_model_inputs():
    '''Define model inputs'''
    
    #Resert the default graph 
    tf.reset_default_graph()
    #Model's placeholders for inputs
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return inputs,targets,keep_prob

In [196]:
def  build_RNN(vocabulary_size,embedding_size,inputs,num_hidden,lstm_layer_numbers,keep_prob,batch_size):
    '''Build RNN'''

    #Embedding Layer
    '''Intialize embeddings for the words. Embedding layer connects the words to the LSTM layers (words are embedded to the embedding_size vectors instead of vocabulary size vectors or one hot vectors). Here, provided by tensorflow, we used random_uniform distribution to create embeddings'''
    #embedding = tf.Variable(tf.random_uniform((vocabulary_size, embedding_size), -1, 1))
    #embed = tf.nn.embedding_lookup(embedding, inputs)
    embed = tf.contrib.layers.embed_sequence(inputs, vocabulary_size, embedding_size)

    #Define LSTM layers
    lstms=[]
    for i in range(lstm_layer_numbers):
        lstms.append(tf.contrib.rnn.BasicLSTMCell(num_hidden))
    # Add regularization dropout to the LSTM cells
    drops = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob) for lstm in lstms]
    # Stack up multiple LSTM layers
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(drops)
    # Getting the initial state
    initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
    outputs, final_state = tf.nn.dynamic_rnn(stacked_lstm, embed, initial_state=initial_state)
        
    return initial_state, outputs, final_state

In [197]:
def get_batches(text_len,seq_len, batch_size):
    '''Using generator to return batches'''
    number_of_words_in_all_batches= seq_len*batch_size
    
    n_batches = text_len//number_of_words_in_all_batches
    text_all_batches = text_words[:n_batches*batch_size]
    print(type(text_all_batches))
    output_seq=[val for i,val in enumerate(text_all_batches) if i%11==0]
    input_seq=[item for item in text_all_batches if item not in output_seq]
    for ii in range(0, text_len, batch_size):
        yield input_seq[ii:ii+batch_size], output_seq[ii:ii+batch_size/seq_len]

In [198]:
#Define Parameters
#Vocab size plus one for 0, the int number that added for padding
n_input = len(word_index)+1
# number of units
num_hidden = 256
lstm_layer_numbers=1
embed_size=300
batch_size= 250
learning_rate=0.001

In [199]:
inputs,targets,keep_prob=create_model_inputs()
initial_state, outputs, final_state = build_RNN(n_input,embed_size,inputs,num_hidden,lstm_layer_numbers,keep_prob,batch_size)
# Loss and optimizer
logits = tf.contrib.layers.fully_connected(outputs, n_input, activation_fn=None)


probs = tf.nn.softmax(logits, name='probs')
cost =  tf.contrib.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([batch_size * seq_len, n_input]),
    
    )                                   
print(logits.shape)
                                                      
optimizer = tf.train.AdamOptimizer(learning_rate)
# Gradient clipping to avoid exploding gradients
gradients = optimizer.compute_gradients(cost)
capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
train_op = optimizer.apply_gradients(capped_gradients) 


#Execute the graph
sess = tf.Session()
saver = tf.train.Saver()
init_op = tf.initialize_all_variables()
sess.run(init_op)

epochs = 35
for epoch in range(epochs):
    state = sess.run(initial_state)
    avg_cost_train = 0 
    avg_acc_train= 0
    for ii, (x, y) in enumerate(get_batches(text_len,seq_len,batch_size), 1):
        print(x)
        print(y)
        print(np.array(y).shape,np.array(x).shape)
        #need to reshape y to feed it to targets
        y = np.array(y).reshape((np.array(np.array(y)).shape)[0], 1)
        state, cost, _= sess.run([final_state, cost,train_op], feed_dict={inputs: x,
                                                        targets: y,keep_prob: 0.5,initial_state: state})
        
        avg_cost_train += cost / no_of_batches_train
        avg_acc_train += acc / no_of_batches_train
    print("acc_train=", avg_acc_train, "acc_val=", avg_acc_val) 
#Save the model into a file 
checkpoint="./model/savedmodel.ckpt"
save_path = saver.save(sess, checkpoint)
sess.close()

(250, ?, 11126)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
<type 'list'>
['potter', "sorcerer's", 'stone', 'chapter', 'dursley,', 'four,', 'drive,', 'say', 'normal,', 'thank', "you'd", 'expect', 'involved', 'anything', 'strange', 'or', 'mysterious,', 'hold', 'with', 'nonsense.', 'director', 'firm', 'called', 'which', 'drills.', 'big,', 'beefy', 'with', 'any', 'neck,', 'although', 'mustache.', 'thin', 'blonde', 'twice', 'amount', 'neck,', 'which', 'useful', 'much', 'time', 'craning', 'garden', 'fences,', 'spying', 'neighbors.', 'son', 'called', 'opinion', 'finer', 'anywhere.', 'everything', 'wanted,', 'also', 'secret,', 'greatest', 'fear', 'somebody', 'discover', 'it.', 'anyone', 'potters.', 'potter', "dursley's", 'sister,', "hadn't", 'several', 'years;', 'fact,', 'pretended', 'sister,', 'sister', 'good-for-nothing', 'husband', 'undursleyish', 'possible', 'be.', 'shuddered', 'neighbors', 'say', 'potters', 'street.', 'potters', 'son,', 'too,', 'never', 'rea

ValueError: invalid literal for long() with base 10: 'potter'